In [169]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, Data
import numpy as np 
import os
from tqdm import tqdm

import torch.nn.functional as F 
from torch.nn import Linear, BatchNorm1d, ModuleList
from torch_geometric.nn import TransformerConv, TopKPooling 
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
torch.manual_seed(42)

In [170]:
Data_Path = "/Users/mpir0002/Downloads/gnn-project/data/raw/HIV.csv"
data = pd.read_csv(Data_Path)
data.head

<bound method NDFrame.head of                                                   smiles activity  HIV_active
0      CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...       CI           0
1      C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...       CI           0
2                       CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21       CI           0
3        Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1       CI           0
4                                 O=S(=O)(O)CCS(=O)(=O)O       CI           0
...                                                  ...      ...         ...
41122  CCC1CCC2c3c([nH]c4ccc(C)cc34)C3C(=O)N(N(C)C)C(...       CI           0
41123  Cc1ccc2[nH]c3c(c2c1)C1CCC(C(C)(C)C)CC1C1C(=O)N...       CI           0
41124  Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C...       CI           0
41125  Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)...       CI           0
41126  CCCCCC=C(c1cc(Cl)c(OC)c(-c2nc(C)no2)c1)c1cc(Cl...       CI           0

[41127 rows x 3 columns]>

In [171]:
class MoleculeDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(MoleculeDataset, self).__init__(root, transform, pre_transform)
        
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            # Get node features
            node_feats = self._get_node_features(mol_obj)
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            # Get labels info
            label = self._get_labels(mol["HIV_active"])

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol["smiles"]
                        ) 
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

    def _get_node_features(self, mol):
        
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        
        all_node_feats = []

        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())

            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))   
        return data


In [64]:
dataset = MoleculeDataset(root='data/' , filename="HIV.csv")

Processing...
 85%|███████████████████████████████████████████████████████████████████████████████████████▊               | 35060/41127 [00:24<00:04, 1331.33it/s][10:59:12] WARNING: not removing hydrogen atom without neighbors
[10:59:12] WARNING: not removing hydrogen atom without neighbors
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 41127/41127 [00:29<00:00, 1394.25it/s]
Done!


In [172]:
dataset

MoleculeDataset(41127)

In [173]:
print(dataset[0].edge_index.t())
print(dataset[0].x)
print(dataset[0].edge_attr)
print(dataset[0].y)

tensor([[ 0,  1],
        [ 1,  0],
        [ 1,  2],
        [ 2,  1],
        [ 2,  3],
        [ 3,  2],
        [ 3,  4],
        [ 4,  3],
        [ 4,  5],
        [ 5,  4],
        [ 4,  6],
        [ 6,  4],
        [ 6,  7],
        [ 7,  6],
        [ 7,  8],
        [ 8,  7],
        [ 8,  9],
        [ 9,  8],
        [ 9, 10],
        [10,  9],
        [10, 11],
        [11, 10],
        [11, 12],
        [12, 11],
        [12, 13],
        [13, 12],
        [13, 14],
        [14, 13],
        [ 7, 15],
        [15,  7],
        [15, 16],
        [16, 15],
        [16, 17],
        [17, 16],
        [17, 18],
        [18, 17],
        [18, 19],
        [19, 18],
        [18, 20],
        [20, 18],
        [17, 21],
        [21, 17],
        [21, 22],
        [22, 21],
        [21,  1],
        [ 1, 21],
        [16,  3],
        [ 3, 16],
        [13,  8],
        [ 8, 13]])
tensor([[ 6.,  1.,  0.,  4.,  0.,  3.,  0.,  0.,  0.],
        [ 6.,  3.,  0.,  3.,  1.,  0.,  0., 

In [174]:
print(len(dataset))
print(dataset.num_classes)

41127
2


In [175]:
dataset.num_node_features

9

In [176]:
data = dataset[0]
print(data.is_directed())

False


In [177]:
print(data.num_nodes)
print(data.num_edges)

23
50


In [178]:
from torch_geometric.nn import GATConv

In [228]:
############## Configuring models architecture ###############

class GNN(torch.nn.Module):
    def __init__(self, feature_size):
        super(GNN, self).__init__()
        
        num_classes = 2
        embedding_size = 1024
        
        # GNN layers
        self.conv1 = GATConv(feature_size , embedding_size , heads=3 , dropout = 0.3)
        self.head_transform1 = Linear(embedding_size*3 , embedding_size)
        self.pool1 = TopKPooling(embedding_size , ratio = 0.8)
        self.conv2 = GATConv(embedding_size , embedding_size , heads=3 , dropout = 0.3)
        self.head_transform2 = Linear(embedding_size*3 , embedding_size)
        self.pool2 = TopKPooling(embedding_size , ratio = 0.5)
        self.conv3 = GATConv(embedding_size , embedding_size , heads=3 , dropout = 0.3)
        self.head_transform3 = Linear(embedding_size*3 , embedding_size)
        self.pool3 = TopKPooling(embedding_size , ratio = 0.3)
        
        # Linear layers
        self.linear1 = Linear(embedding_size*2 , 1024)
        self.linear2 = Linear(1024 , num_classes)
        
    def forward(self , x , edge_index , batch_index):
        
        x = self.conv1(x , edge_index)
        x = self.head_transform1(x)
        x , edge_index , edge_attr , batch_index , _ , _ = self.pool1(x , edge_index, None, batch_index)
        x1 = torch.cat([gmp(x , batch_index) , gap(x , batch_index)] , dim = 1)
        
        x = self.conv2(x , edge_index)
        x = self.head_transform2(x)
        x , edge_index , edge_attr , batch_index , _ , _ = self.pool2(x , edge_index, None, batch_index)
        x2 = torch.cat([gmp(x , batch_index) , gap(x , batch_index)] , dim = 1)
        
        x = self.conv3(x , edge_index)
        x = self.head_transform3(x)
        x , edge_index , edge_attr , batch_index , _ , _ = self.pool3(x , edge_index, None, batch_index)
        x3 = torch.cat([gmp(x , batch_index) , gap(x , batch_index)] , dim = 1)
        
        x = x1 + x2 + x3
        
        x = self.linear1(x).relu()
        x = F.dropout(x , p = 0.5 , training = self.training)
        x = self.linear2(x)
        
        return x

In [180]:
len(dataset)

41127

In [242]:
### Spliting data into train and test

torch.manual_seed(1234)
dataset = dataset.shuffle()

# Once it's shuffled, we slice the data to split
train_dataset = dataset[0:300]
test_dataset = dataset[301:400]

# Take a look at the training versus test graphs
print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 300
Number of test graphs: 99


In [243]:
########### Training Data ###############

model = GNN(feature_size=train_dataset[0].x.shape[1])
model

GNN(
  (conv1): GATConv(9, 1024, heads=3)
  (head_transform1): Linear(in_features=3072, out_features=1024, bias=True)
  (pool1): TopKPooling(1024, ratio=0.8, multiplier=1.0)
  (conv2): GATConv(1024, 1024, heads=3)
  (head_transform2): Linear(in_features=3072, out_features=1024, bias=True)
  (pool2): TopKPooling(1024, ratio=0.5, multiplier=1.0)
  (conv3): GATConv(1024, 1024, heads=3)
  (head_transform3): Linear(in_features=3072, out_features=1024, bias=True)
  (pool3): TopKPooling(1024, ratio=0.3, multiplier=1.0)
  (linear1): Linear(in_features=2048, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=2, bias=True)
)

In [244]:
weights = torch.tensor([1,10] , dtype = torch.float32)
loss_fn = torch.nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.SGD(model.parameters() , lr = 0.1 , momentum = 0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer , gamma = 0.95)

In [245]:
train_loader = DataLoader(train_dataset , batch_size=40 , shuffle=True)
test_loader = DataLoader(test_dataset , batch_size = 40 , shuffle=True)

In [246]:

def calculate_metrics(y_pred, y_true, epoch, type):
    print(f"\n Confusion matrix: \n {confusion_matrix(y_pred, y_true)}")
    print(f"F1 Score: {f1_score(y_true, y_pred)}")
    print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    print(f"Precision: {prec}")
    print(f"Recall: {rec}")

In [250]:
############ Model evaluation ##################

def train(epoch , model, train_loader, loss_fn):
    
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    
    for _, batch in enumerate(tqdm(train_loader)):
         
        # Reset gradients
        optimizer.zero_grad() 
        # Passing the node features and the connection info
        pred = model(batch.x.float(), 
                                batch.edge_index, 
                                batch.batch) 
        # Calculating the loss and gradients
        loss = torch.sqrt(loss_fn(pred, batch.y))
        loss.backward()  
        optimizer.step()  
        # Update tracking
        running_loss += loss.item()
        step += 1
        
        all_preds.append(np.argmax(pred.detach().cpu().numpy(), axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    calculate_metrics(all_preds, all_labels, epoch, "train")
    return running_loss

def test(epoch, model, test_loader, loss_fn):
    
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    
    for batch in test_loader:
        pred = model(batch.x.float(), 
                        batch.edge_attr.float(),
                        batch.batch) 
        loss = torch.sqrt(loss_fn(pred, batch.y))

         # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds = np.concatenate(all_preds).ravel()
        all_labels = np.concatenate(all_labels).ravel()
    
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    print(all_preds[:10])
    print(all_labels[:10])
    calculate_metrics(all_preds, all_labels, epoch, "test")
    #log_conf_matrix(all_preds, all_labels, epoch)
    return running_loss

In [251]:
train_dataset

MoleculeDataset(300)

In [252]:
train(epoch= 1 , model=model , train_loader=train_dataset , loss_fn=loss_fn)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:18<00:00, 16.56it/s]


 Confusion matrix: 
 [[288  12]
 [  0   0]]
F1 Score: 0.0
Accuracy: 0.96
Precision: 0.0
Recall: 0.0



/Users/mpir0002/miniconda3/envs/tf/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


nan

In [258]:


loss_fn = torch.nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


def train(epoch, model, train_loader, loss_fn):
    model.train()
    running_loss = 0.0
    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()

        # Forward pass
        pred = model(batch.x.float(), batch.edge_index, batch.batch)
        
        # Calculate the loss
        loss = loss_fn(pred, batch.y)

        # Backward pass
        loss.backward()

        # Update parameters
        optimizer.step()

        running_loss += loss.item()

    # Print or log the training loss
    print(f"Epoch {epoch}, Loss: {running_loss / len(train_loader)}")

# Example usage:
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    train(epoch, model, train_dataset, loss_fn)


Epoch 1, Loss: nan
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
